In [23]:
import numpy as np
import cv2

In [24]:
# read video
cap = cv2.VideoCapture('../input/movie/raw/bacteria.mp4')

# algorithm
model_dict = { 
    'CNT': cv2.bgsegm.createBackgroundSubtractorCNT(), 
    'CMG': cv2.bgsegm.createBackgroundSubtractorGMG(),
    'GSOC': cv2.bgsegm.createBackgroundSubtractorGSOC(),
    'LSBP': cv2.bgsegm.createBackgroundSubtractorLSBP(),
    'MOG': cv2.bgsegm.createBackgroundSubtractorMOG(),
    'KNN': cv2.createBackgroundSubtractorKNN(),
    'MOG2': cv2.createBackgroundSubtractorMOG2()
}

# config
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = cap.get(cv2.CAP_PROP_FPS)
fourcc = cv2.VideoWriter_fourcc(*'MP4V')
writer = cv2.VideoWriter(
    '../output/movie/output_contours.mp4',
    fourcc,
    fps,
    (width, height),
    isColor = True
)

while True:
    ret, frame = cap.read()
    if not ret:
        break

    model = model_dict['KNN']

    mask = model.apply(frame)

    # binarization using Otsu's method
    _, th = cv2.threshold(mask,0,255,cv2.THRESH_BINARY+cv2.THRESH_OTSU)

    # configure the kernel
    kernel = np.ones((20,20),np.uint8)

    # morphological transformation(Closing)
    th_closing = cv2.morphologyEx(th, cv2.MORPH_CLOSE, kernel)

    # contour extraction
    contours, hierarchy = cv2.findContours(th_closing,
                                           cv2.RETR_LIST,
                                           cv2.CHAIN_APPROX_NONE)

    # draw the contours on the source image
    img_contour = cv2.drawContours(frame.copy(), contours, -1, (0, 255, 0), 2)

    # コロニーの輪郭を抽出
    area_list = []

    for i in range(len(contours)):
        if len(contours[i]) > 0:
            
            # 小さい輪郭を除去
            if cv2.contourArea(contours[i]) < 50:
                continue
            
            # コロニーリストに格納
            area = cv2.contourArea(contours[i])
            area_list.append(area)    

    cv2.putText(
        img_contour,
        text=f'Number of Colony: {len(area_list)}',
        org=(20, 20),
        fontFace=cv2.FONT_HERSHEY_SIMPLEX,
        fontScale=0.5,
        color=(0, 0, 255),
        thickness=1,
        lineType=cv2.LINE_4
    )                                  

    cv2.imshow('Colony Count', img_contour)

    writer.write(img_contour)

    k = cv2.waitKey(30) & 0xff
    if k == 27:
        break

cap.release()
writer.release()
cv2.destroyAllWindows()